## Run many Batch Normalization experiments using Cloud using ML Engine

In [5]:
# change these to try this notebook out
BUCKET = 'crawles-sandbox'  # change this to your GCP bucket
PROJECT = 'crawles-sandbox'  # change this to your GCP project
REGION = 'us-central1'

# Import os environment variables
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

Let’s test how Batch Normalization impacts models of varying depths. We can launch many experiments in parallel using Google Cloud ML Engine. We will fire off 14 jobs with varying hyperparameters:

* With and without Batch Normalization
* Varying model depths from 1 hidden layer to 7 hidden layers

We use the [tf.estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator) API to build a model and deploy it using [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/technical-overview).

In [6]:
!ls mnist_classifier/
!ls mnist_classifier/trainer/

PKG-INFO  setup.cfg setup.py  trainer
__init__.py __pycache__ model.py    task.py


In [7]:
%%bash
submitMLEngineJob() {
    gcloud ml-engine jobs submit training $JOBNAME \
        --package-path=$(pwd)/mnist_classifier/trainer \
        --module-name trainer.task \
        --region $REGION \
        --staging-bucket=gs://$BUCKET \
        --scale-tier=BASIC \
        --runtime-version=1.4 \
        -- \
        --outdir $OUTDIR \
        --hidden_units $net \
        --num_steps 10 \
        $batchNorm
}

# submit for different layer sizes
export PYTHONPATH=${PYTHONPATH}:${PWD}/mnist_classifier
for batchNorm in '' '--use_batch_normalization'
do
    net=''
    for layer in 500 400 300 200 100 50 25;
    do
        net=$net$layer
        netname=${net//,/_}${batchNorm/--use_batch_normalization/_bn}
        echo $netname
        JOBNAME=mnist$netname_$(date -u +%y%m%d_%H%M%S)
        OUTDIR=gs://${BUCKET}/mnist_models/mnist_model$netname/trained_model
        echo $OUTDIR $REGION $JOBNAME
        gsutil -m rm -rf $OUTDIR
        submitMLEngineJob
        net=$net,    
    done 
done

500
gs://crawles-sandbox/mnist_models/mnist_model500/trained_model us-central1 mnist180225_230728
jobId: mnist180225_230728
state: QUEUED
500_400
gs://crawles-sandbox/mnist_models/mnist_model500_400/trained_model us-central1 mnist180225_230739
jobId: mnist180225_230739
state: QUEUED
500_400_300
gs://crawles-sandbox/mnist_models/mnist_model500_400_300/trained_model us-central1 mnist180225_230745
jobId: mnist180225_230745
state: QUEUED
500_400_300_200
gs://crawles-sandbox/mnist_models/mnist_model500_400_300_200/trained_model us-central1 mnist180225_230752
jobId: mnist180225_230752
state: QUEUED
500_400_300_200_100
gs://crawles-sandbox/mnist_models/mnist_model500_400_300_200_100/trained_model us-central1 mnist180225_230759
jobId: mnist180225_230759
state: QUEUED
500_400_300_200_100_50
gs://crawles-sandbox/mnist_models/mnist_model500_400_300_200_100_50/trained_model us-central1 mnist180225_230805
jobId: mnist180225_230805
state: QUEUED
500_400_300_200_100_50_25
gs://crawles-sandbox/mnist_m

Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/#1519593806088932...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/checkpoint#1519593807030273...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/eval/#1519593804113533...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/eval/events.out.tfevents.1519593804.cmle-training-9796021764610942655#1519593808912134...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/events.out.tfevents.1519593798.cmle-training-9796021764610942655#1519593807708461...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/graph.pbtxt#1519593799729749...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/model.ckpt-1.data-00000-of-00001#1519593801454038...
Removing gs://crawles-sandbox/mnist_models/mnist_model500/trained_model/model.ckpt-1.index#1519593801670527...
Removing gs://crawles-sandbox/mnist_models/mnis